In [1]:
!git clone https://github.com/aim-hackathon-2020/pump_prediction.git

fatal: destination path 'pump_prediction' already exists and is not an empty directory.


In [0]:
# importing our favorite libraries
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score

### Load Cleaned Data and Create Model Input Format

In [3]:
train = pd.read_csv('clean.csv')
test = pd.read_csv('clean_test.csv')
print("Train Set: ",train.shape, " Test Set: ", test.shape)

Train Set:  (59400, 21)  Test Set:  (14850, 20)


In [0]:
resam = train.copy()
x1 = train[train["status_group"] == "functional needs repair"]
x2 = train[train["status_group"] == "functional"]
x3 = train[train["status_group"] == "non functional"]
x2 = x2.sample(frac=1).reset_index(drop=True)
x3 = x3.sample(frac=1).reset_index(drop=True)
undersampled = pd.concat([x1, x2.head(4317), x3.head(4317)])

In [5]:
#drop this unnamed column 
target=train.pop('status_group')
train['train']=1
test['train']=0
combined = pd.concat([train, test])
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### MODEL SELECTION

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

#define X train
X = train_df
#y_train indentified at the beginning 
y = target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)
y.value_counts(normalize=True)*100

functional                 54.308081
non functional             38.424242
functional needs repair     7.267677
Name: status_group, dtype: float64

In [0]:
cls_weights = {0:(54.30/7.26), 2:(38.42/7.26), 1:1.0}

In [8]:
%%time
from sklearn.ensemble import RandomForestClassifier
Rforest = RandomForestClassifier(n_estimators=1000)
Rforest.fit(X_train,y_train)

CPU times: user 1min 20s, sys: 967 ms, total: 1min 21s
Wall time: 1min 21s


In [9]:
print(f1_score(y_test,Rforest.predict(X_test), average='weighted'))
print(accuracy_score(y_test,Rforest.predict(X_test)))

0.8010022340484592
0.8066523824099582


In [10]:
%%time
#Algos
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
TunedRForest = RandomForestClassifier(criterion='entropy', n_estimators=1000)
TunedRForest.fit(X_train,y_train)
print(f1_score(y_test,TunedRForest.predict(X_test), average='weighted'))
print(accuracy_score(y_test,TunedRForest.predict(X_test)))

0.8013631062654931
0.8071625344352618
CPU times: user 2min 15s, sys: 803 ms, total: 2min 16s
Wall time: 2min 16s


In [11]:
%%time
#Fitting Classifier to Training Set. Create a classifier object here and call it classifierObj 
ETCLassifier = ExtraTreesClassifier(criterion='entropy') 
ETCLassifier.fit(X_train,y_train)
print(f1_score(y_test,ETCLassifier.predict(X_test), average='weighted'))
print(accuracy_score(y_test,ETCLassifier.predict(X_test)))

0.7889950630287231
0.7927762473217018
CPU times: user 7.21 s, sys: 213 ms, total: 7.42 s
Wall time: 7.43 s


In [12]:
%%time
from xgboost import XGBClassifier
xgb = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', 
                      num_class = 3, maximize = False, eval_metric = 'merror', eta = .2,
                      max_depth = 14, colsample_bytree = .4)
xgb.fit(X_train,y_train)
print(f1_score(y_test,xgb.predict(X_test), average='weighted'))
print(accuracy_score(y_test,xgb.predict(X_test)))

0.8049343227520531
0.8133863891439649
CPU times: user 30.5 s, sys: 66.9 ms, total: 30.5 s
Wall time: 30.6 s


In [13]:
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
TunedBaggingC = BaggingClassifier(DecisionTreeClassifier(random_state=2, criterion='entropy', max_depth= 20))
TunedBaggingC.fit(X_train,y_train)
print(f1_score(y_test,TunedBaggingC.predict(X_test), average='weighted'))
print(accuracy_score(y_test,TunedBaggingC.predict(X_test)))

0.7882260284936023
0.7963473114988266


In [14]:
%%time
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
	estimators = [('RForest', Rforest), ('xgboost', xgb)],
	voting = 'hard')
voting_clf.fit(X_train, y_train)
print(f1_score(y_test,voting_clf.predict(X_test), average='weighted'))
print(accuracy_score(y_test,voting_clf.predict(X_test)))

0.8030453478280732
0.8125191307009488
CPU times: user 2min 2s, sys: 844 ms, total: 2min 3s
Wall time: 2min 3s


In [0]:
test_df_feat = test_df.copy()

In [0]:
a=test_df_feat.pop('id')

In [23]:
submission = pd.DataFrame.from_dict({
'id': a,
'status_group': Rforest.predict(test_df_feat)})
submission.status_group.value_counts(normalize=True)*100

functional                 62.269360
non functional             33.959596
functional needs repair     3.771044
Name: status_group, dtype: float64

In [18]:

'''
RFOREST:

functional                 62.390572
non functional             33.872054
functional needs repair     3.737374

functional                 62.269360
non functional             33.959596
functional needs repair     3.771044

XGB:

functional                 65.319865
non functional             32.000000
functional needs repair     2.680135

'''

'\nRFOREST:\n\nfunctional                 62.390572\nnon functional             33.872054\nfunctional needs repair     3.737374\n\nXGB:\n\nfunctional                 65.319865\nnon functional             32.000000\nfunctional needs repair     2.680135\n\n'

In [0]:
submission.to_csv("submission.csv", index=False)